In [2]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
import os 


def Normlize(Z):
    Zmax, Zmin = Z.max(axis=1), Z.min(axis=1)
    Zmean = Z.mean(axis=1)
    #按列排序
    Zmax, Zmin = Zmax.reshape(-1, 1), Zmin.reshape(-1, 1)
    Zmean = Zmean.reshape(-1, 1)
    Z = (Z - Zmean) / (Zmax - Zmin)
    return Z


def Data_Reading(Normalization=True):
    train_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainset.npy')
    train_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainlabel.npy')

    test_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testset.npy')
    test_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testlabel.npy')

    # Normalization
    train_x_Normed = Normlize(train_x)

    test_x_Normed = Normlize(test_x)

    # xlsx to tensor
    if Normalization:
        train_x = torch.from_numpy(train_x_Normed).type(torch.cuda.FloatTensor)

        test_x = torch.from_numpy(test_x_Normed).type(torch.cuda.FloatTensor)

        train_y = torch.from_numpy(train_y).type(torch.cuda.int64)

        test_y = torch.from_numpy(test_y).type(torch.cuda.int64)

    else:
        train_x = torch.from_numpy(train_x).type(torch.FloatTensor)

        test_x = torch.from_numpy(test_x).type(torch.FloatTensor)

        train_y = torch.from_numpy(train_y).type(torch.int64)

        test_y = torch.from_numpy(test_y).type(torch.int64)
    # reshape
    train_x = train_x.view(408, 1, 10, 120)
    test_x = test_x.view(202, 1, 10, 120)

    return train_x, test_x, train_y, test_y


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,(4,4),stride=(2,2))#(10 - 4)/2 + 1 = 4   (120 - 4)/2 + 1 = 59 
        self.conv2 = nn.Conv2d(6,10,(3,3),stride=(1,2))#(4 - 3)/1 + 1 = 2   (59 - 3)/2 + 1 = 29 
        self.conv3 = nn.Conv2d(10,14,(2,2),stride=(1,1))#(2 - 2)/1 + 1 = 1  (29 - 2)/1 + 1 = 28
        self.classes = [0, 1, 2, 3]
        # self.conv1 = nn.Conv2d(1,6,(10,3),stride=(1,3))
        # self.conv2 = nn.Conv2d(6,10,(1,3),stride=(1,2))
        self.fc = nn.Linear(14*1*28,4)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.xavier_uniform_(m.weight)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.01)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1) # flatten the tensor
        x = self.fc(x)
        # return x
        return F.log_softmax(x,dim=0)


#training
batch_size = 17
def train(train_x,train_y,step=10):
    for epoch in range(50):
        running_loss = 0.0
        total = 0.0
        correct = 0.0
        for i in range(0,(int)(len(train_x)/batch_size)):
            t_x = Variable(train_x[i*batch_size:i*batch_size+batch_size])
            t_y = Variable(train_y[i*batch_size:i*batch_size+batch_size])
            out = cnn(t_x)
            #forward
            #loss_func = nn.MSELoss() 均方损失函数  loss(x(i),y(i)) = (x(i) - y(i))^2
            loss = loss_func(out, t_y)#.long().squeeze()
            #梯度初始化为零
            optimizer.zero_grad()
             
            #backward
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(out.data, 1)
            # print(predicted)
            total += t_y.size(0)
            # print(total)
            correct += predicted.eq(t_y.data).cpu().sum()
            
        running_loss = running_loss / 24

        # if (epoch + 1) % step == 0:
        print('Epoch[{}], loss: {:.12f}, correct:{}'.format(epoch + 1, running_loss, correct))
        

def predict(test_x,test_y):
    test_acc = 0.0

    te_x = Variable(test_x)
    tey1 = Variable(test_y)
    out1 = cnn(te_x)
    loss = loss_func(out1, tey1)

    _, predicted = torch.max(out1.data, 1)
    print(predict)

    print('loss:{}, out:{}, tabel:{}'.format(loss, out1, tey1))


if __name__ == '__main__':
    device = torch.device("cpu" if torch.cuda.is_available() else "cpu")#cuda:0

    print(device)

    cnn = Net()
    print(cnn)
    cnn.to(device)

    #sgd -> stochastic gradient descent
    optimizer = optim.SGD(cnn.parameters(), lr=0.0001, momentum=0.8)
    # loss_func = nn.MSELoss()
    loss_func = nn.CrossEntropyLoss()

    train_x, test_x, train_y, test_y = Data_Reading(Normalization=0)
    train_y = train_y.squeeze()
    test_y = test_y.squeeze()
    train_x = train_x.to(device)
    test_x = test_x.to(device)
    train_y = train_y.to(device)
    test_y = test_y.to(device)
    print(train_y)

    train(train_x, train_y, step=10)
    # predict(test_x,test_y)

cpu
Net(
  (conv1): Conv2d(1, 6, kernel_size=(4, 4), stride=(2, 2))
  (conv2): Conv2d(6, 10, kernel_size=(3, 3), stride=(1, 2))
  (conv3): Conv2d(10, 14, kernel_size=(2, 2), stride=(1, 1))
  (fc): Linear(in_features=392, out_features=4, bias=True)
)
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2